In [1]:

%pprint
%matplotlib inline
import sys
import os.path as osp
import os

executable_path = sys.executable
scripts_folder = osp.join(osp.dirname(executable_path), 'Scripts'); assert osp.exists(scripts_folder)
py_folder = osp.abspath(osp.join(os.pardir, 'py')); assert osp.exists(py_folder), "Create the py folder"
ffmpeg_folder = r'C:\ffmpeg\bin'; assert osp.exists(ffmpeg_folder)
shared_folder = osp.abspath(osp.join(os.pardir, os.pardir, 'share')); assert osp.exists(shared_folder)

if (scripts_folder not in sys.path): sys.path.insert(1, scripts_folder)
if (py_folder not in sys.path): sys.path.insert(1, py_folder)
if (ffmpeg_folder not in sys.path): sys.path.insert(1, ffmpeg_folder)
if shared_folder not in sys.path: sys.path.insert(1, shared_folder)

from notebook_utils import NotebookUtilities
nu = NotebookUtilities(
    data_folder_path=osp.abspath(osp.join(os.pardir, 'data')),
    saves_folder_path=osp.abspath(osp.join(os.pardir, 'saves'))
)

# Import needed libraries
from bs4 import BeautifulSoup as bs
from pandas import DataFrame
import ast

Pretty printing has been turned OFF


In [2]:


bookmark_files = []
common_bookmark_names = [
    'Bookmarks',          # Chrome, Edge
    'bookmarks.html',     # Firefox (exported)
    'safari_bookmarks.html',  # Safari (exported)
    'Bookmarks.plist',    # Safari (plist file)
    'places.sqlite'       # Firefox (database file)
]
for start_directory in ['C:\\', 'E:\\']:
    for parent_directory, child_directories, files in os.walk(start_directory):
        for file in files:
            if file in common_bookmark_names:
                bookmark_files.append(os.path.join(parent_directory, file))

In [3]:

bookmark_files

['C:\\tmp\\playwright\\places.sqlite', 'C:\\Users\\daveb\\AppData\\Local\\Google\\Chrome\\User Data\\Default\\Bookmarks', 'C:\\Users\\daveb\\AppData\\Local\\Google\\Chrome\\User Data\\Profile 1\\Bookmarks', 'C:\\Users\\daveb\\AppData\\Local\\Google\\Chrome\\User Data\\Profile 2\\Bookmarks', 'C:\\Users\\daveb\\AppData\\Local\\Google\\Chrome\\User Data\\Snapshots\\126.0.6478.185\\Default\\Bookmarks', 'C:\\Users\\daveb\\AppData\\Local\\Google\\Chrome\\User Data\\Snapshots\\126.0.6478.185\\Profile 1\\Bookmarks', 'C:\\Users\\daveb\\AppData\\Local\\Google\\Chrome\\User Data\\Snapshots\\127.0.6533.120\\Default\\Bookmarks', 'C:\\Users\\daveb\\AppData\\Local\\Google\\Chrome\\User Data\\Snapshots\\127.0.6533.120\\Profile 1\\Bookmarks', 'C:\\Users\\daveb\\AppData\\Local\\Google\\Chrome\\User Data\\Snapshots\\130.0.6723.117\\Default\\Bookmarks', 'C:\\Users\\daveb\\AppData\\Local\\Google\\Chrome\\User Data\\Snapshots\\130.0.6723.117\\Profile 1\\Bookmarks', 'C:\\Users\\daveb\\AppData\\Local\\Microso

In [10]:

import sqlite3
import pandas as pd

rows_list = []
for bookmark_path in bookmark_files:
    if bookmark_path.endswith('.sqlite'):
        conn = sqlite3.connect(bookmark_path)
        tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
        for table_name in tables.name:
            table_name_data = pd.read_sql_query(f"SELECT * FROM {table_name};", conn)
            if table_name_data.shape[0]:
                contains_http = False
                for column in table_name_data.select_dtypes(include=['object']).columns:
                    if table_name_data[column].str.startswith('http').any():
                        contains_http = True
                        break
                if contains_http:
                    row_dict = {
                        'bookmark_path': bookmark_path,
                        'bookmark_name': osp.basename(bookmark_path),
                        'table_name': table_name,
                        'table_column': column,
                    }
                    rows_list.append(row_dict)
        conn.close()
bookmark_columns_df = DataFrame(rows_list)

In [12]:

bookmark_columns_df.groupby('table_name').size()

table_name
moz_bookmarks        1
moz_inputhistory     1
moz_origins         19
moz_places          19
dtype: int64

In [14]:

mask_series = bookmark_columns_df.table_name.isin(['moz_bookmarks', 'moz_inputhistory'])
for _, row_series in bookmark_columns_df[mask_series].iterrows():
    bookmark_path = row_series.bookmark_path
    table_name = row_series.table_name
    print(table_name)
    conn = sqlite3.connect(bookmark_path)
    table_name_df = pd.read_sql_query(f"SELECT * FROM {table_name};", conn)
    display(table_name_df)
    conn.close()

moz_inputhistory


,place_id,input,use_count
0,15672,im,0.122334
1,15672,imgur,0.147057
2,6468,geni.com,0.184753
3,20058,firs,0.161560
4,1463,turbo,0.188064
5,169,turbo,0.188064
6,4428,old.,0.375880
7,9087,python round float to,0.320043
8,4428,old,2.184488
9,20658,alight,0.422822


moz_bookmarks


,id,type,fk,parent,position,title,keyword_id,folder_type,dateAdded,lastModified,guid,syncStatus,syncChangeCounter
0,1,2,NaN,0,0,,None,None,1645056347488000,1677789729327000,root________,2,1
1,2,2,NaN,1,0,menu,None,None,1645056347488000,1659903526644000,menu________,2,0
2,3,2,NaN,1,1,toolbar,None,None,1645056347488000,1677789729327000,toolbar_____,2,0
3,4,2,NaN,1,2,tags,None,None,1645056347488000,1668086390799000,tags________,2,3
4,5,2,NaN,1,3,unfiled,None,None,1645056347488000,1648674029536000,unfiled_____,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7664,12034,1,23441.0,12033,0,None,None,None,1667560469120000,1667560469120000,a8504Wy3kcXO,1,2
7665,12035,1,23771.0,102,11,📙 Emojipedia — 😃 Home of Emoji Meanings 💁👌🎍😍,None,None,1668086378935000,1668086397225000,iVh7gNzf77cL,2,0
7666,12036,2,NaN,4,9,emoji,None,None,1668086390799000,1668086390809000,g7SbGMWcA1sc,1,2
7667,12037,1,23771.0,12036,0,None,None,None,1668086390809000,1668086390809000,LT8NAQx1q4YU,1,2


In [ ]:

rows_list = []
for bookmark_path in bookmark_files:
    if osp.basename(bookmark_path) == 'Bookmarks':
        with open(bookmark_path, 'r', encoding='utf-8') as file:
            bookmarks_data = json.load(file)
            display(bookmarks_data)
            break


----